In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup
 

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sidmitta/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#! pip install bs4 # in case you don't have it installed

# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz

## Read Data

In [3]:
dataset = pd.read_csv('amazon_reviews_us_Kitchen_v1_00.tsv', sep='\t', error_bad_lines=False)

b'Skipping line 16148: expected 15 fields, saw 22\nSkipping line 20100: expected 15 fields, saw 22\nSkipping line 45178: expected 15 fields, saw 22\nSkipping line 48700: expected 15 fields, saw 22\nSkipping line 63331: expected 15 fields, saw 22\n'
b'Skipping line 86053: expected 15 fields, saw 22\nSkipping line 88858: expected 15 fields, saw 22\nSkipping line 115017: expected 15 fields, saw 22\n'
b'Skipping line 137366: expected 15 fields, saw 22\nSkipping line 139110: expected 15 fields, saw 22\nSkipping line 165540: expected 15 fields, saw 22\nSkipping line 171813: expected 15 fields, saw 22\n'
b'Skipping line 203723: expected 15 fields, saw 22\nSkipping line 209366: expected 15 fields, saw 22\nSkipping line 211310: expected 15 fields, saw 22\nSkipping line 246351: expected 15 fields, saw 22\nSkipping line 252364: expected 15 fields, saw 22\n'
b'Skipping line 267003: expected 15 fields, saw 22\nSkipping line 268957: expected 15 fields, saw 22\nSkipping line 303336: expected 15 field

b'Skipping line 1968846: expected 15 fields, saw 22\nSkipping line 1999941: expected 15 fields, saw 22\nSkipping line 2001492: expected 15 fields, saw 22\nSkipping line 2011204: expected 15 fields, saw 22\nSkipping line 2025295: expected 15 fields, saw 22\n'
b'Skipping line 2041266: expected 15 fields, saw 22\nSkipping line 2073314: expected 15 fields, saw 22\nSkipping line 2080133: expected 15 fields, saw 22\nSkipping line 2088521: expected 15 fields, saw 22\n'
b'Skipping line 2103490: expected 15 fields, saw 22\nSkipping line 2115278: expected 15 fields, saw 22\nSkipping line 2153174: expected 15 fields, saw 22\nSkipping line 2161731: expected 15 fields, saw 22\n'
b'Skipping line 2165250: expected 15 fields, saw 22\nSkipping line 2175132: expected 15 fields, saw 22\nSkipping line 2206817: expected 15 fields, saw 22\nSkipping line 2215848: expected 15 fields, saw 22\nSkipping line 2223811: expected 15 fields, saw 22\n'
b'Skipping line 2257265: expected 15 fields, saw 22\nSkipping line

## Keep Reviews and Ratings

In [24]:
data = dataset[['review_body', 'star_rating']]

#Printing 3 sample reviews with ratings and count of reviews for each rating
print(data.iloc[:3])

Positive = data['star_rating'][(data['star_rating']>3.0)].count()
Negative = data['star_rating'][(data['star_rating']<3.0)].count()
Neutral = data['star_rating'][(data['star_rating']==3.0)].count()

print("The counts for positive, negative and neutral reviews are: " +str(Positive)+", "+str(Negative)+", "+str(Neutral))

ratings_1 = data['star_rating'][(data['star_rating']==1.0)].count()
print('Number of reviews with rating 1 = ', ratings_1)

ratings_2 = data['star_rating'][(data['star_rating']==2.0)].count()
print('Number of reviews with rating 2 = ', ratings_2)

ratings_3 = data['star_rating'][(data['star_rating']==3.0)].count()
print('Number of reviews with rating 3 = ', ratings_3)

ratings_4 = data['star_rating'][(data['star_rating']==4.0)].count()
print('Number of reviews with rating 4 = ', ratings_4)

ratings_5 = data['star_rating'][(data['star_rating']==5.0)].count()
print('Number of reviews with rating 5 = ', ratings_5)

                                         review_body  star_rating
0                Beautiful.  Looks great on counter.          5.0
1  I personally have 5 days sets and have also bo...          5.0
2  Fabulous and worth every penny. Used for clean...          5.0
The counts for positive, negative and neutral reviews are: 3856492, 668848, 349547
Number of reviews with rating 1 =  426900
Number of reviews with rating 2 =  241948
Number of reviews with rating 3 =  349547
Number of reviews with rating 4 =  731733
Number of reviews with rating 5 =  3124759


# Labelling Reviews:
## The reviews with rating 4,5 are labelled to be 1 and 1,2 are labelled as 0. Discard the reviews with rating 3'

In [5]:
df = data[data['star_rating'] !=3]
df['star_rating'] = df['star_rating'].apply(lambda x: 1 if x > 3 else 0)

/Users/sidmitta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


 ## We select 200000 reviews randomly with 100,000 positive and 100,000 negative reviews.



In [6]:
df1 = df.query(' star_rating == 1').sample(n=100000)
df2 = df.query(' star_rating == 0').sample(n=100000)
frames = [df1, df2]
df = pd.concat(frames)
df = df.sample(frac=1)

# Data Cleaning

## Convert the all reviews into the lower case.

In [7]:
df["review_body"] = df['review_body'].str.lower()

## remove the HTML and URLs from the reviews

In [8]:
total_rows = len(df['review_body'])
len_before_cleaning = df['review_body'].apply(lambda x: len(x) if type(x).__name__== 'str' else 0).mean()

df['review_body'] = df['review_body'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

import re

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, ' ', raw_html)
  return cleantext

df['review_body'] = df['review_body'].apply(lambda x: cleanhtml(x))

,review_body,star_rating
4720200,i wanted a grill for making myself hamburgers ...,0
4208050,i love this water bottle. i bought two at the...,1
1389475,great product. i use this all the time. sturdy...,1
924830,the area great,1
4063055,"we just received these as a gift, but they arr...",0


## remove non-alphabetical characters

In [9]:
df['review_body'].str.replace('[^ a-zA-Z]', '')

4720200    i wanted a grill for making myself hamburgers ...
4208050    i love this water bottle  i bought two at the ...
1389475    great product i use this all the time sturdy a...
924830                                        the area great
4063055    we just received these as a gift but they arri...
4550586    i love this little italian delight  i use it e...
2289117    i have yet to learn how this item works  it is...
3681275    about the best thing i can say about this toas...
1074225    this mat is fantastic i was concerned initiall...
1366179                      nice glasses hand fit is superb
1994496    well my friend r happy with the gifts but it w...
4604381    the glass container really sets this blender a...
704130     the worst pan i have ever owned everything sti...
4676115    ok its another unnecessary kitchen gadget but ...
2434613    had to send it back because it was defective  ...
477506     after a two minute learning curve we are now u...
2954880    i love these 

## Remove the extra spaces between the words

In [10]:
df['review_body'] = df['review_body'].replace('\s+', ' ', regex=True)

,review_body,star_rating
4720200,i wanted a grill for making myself hamburgers ...,0
4208050,i love this water bottle. i bought two at the ...,1
1389475,great product. i use this all the time. sturdy...,1
924830,the area great,1
4063055,"we just received these as a gift, but they arr...",0


## perform contractions on the reviews.

In [11]:
def contractionfunction(phrase):
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

df['review_body'] = df.review_body.apply(lambda row: contractionfunction(row))

len_after_cleaning = df['review_body'].apply(lambda x: len(x) if type(x).__name__== 'str' else 0).mean()
print('Length before and after cleaning text %.3f' %len_before_cleaning,', %.3f'%len_after_cleaning)

Length before and after cleaning text 323.428 , 320.139


# Pre-processing

## remove the stop words 

In [12]:
from nltk.corpus import stopwords
len_before_preprcessing = df['review_body'].apply(lambda x: len(x) if type(x).__name__== 'str' else 0).mean()
stop = stopwords.words('english')
df['review_body'] = df['review_body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

## perform lemmatization  

In [13]:
from nltk.stem import WordNetLemmatizer
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    word_list = nltk.word_tokenize(text)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_output

df['review_body'] = df.review_body.apply(lambda x: lemmatize_text(x))
len_after_preprcessing = df['review_body'].apply(lambda x: len(x) if type(x).__name__== 'str' else 0).mean()
print('Length before and after pre-processing text %.3f' %len_before_preprcessing,', %.3f'%len_after_preprcessing)

Length before and after pre-processing text 320.139 , 213.265


# TF-IDF Feature Extraction

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['review_body'])
train_X, test_X, train_Y, test_Y = train_test_split(X, df['star_rating'],test_size=0.2, random_state=0)

/Users/sidmitta/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Perceptron

In [15]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
clf = Perceptron(tol=1e-3, random_state=0)
clf.fit(train_X, train_Y)

perceptron_pred_train = clf.predict(train_X)
perceptron_pred_test = clf.predict(test_X)

print('%.3f' % accuracy_score(train_Y, perceptron_pred_train),
      '%.3f' % precision_score(train_Y, perceptron_pred_train),
      '%.3f' % recall_score(train_Y, perceptron_pred_train),
      '%.3f' % f1_score(train_Y, perceptron_pred_train),
      '%.3f' % accuracy_score(test_Y, perceptron_pred_test),
      '%.3f' % precision_score(test_Y, perceptron_pred_test),
      '%.3f' % recall_score(test_Y, perceptron_pred_test),
      '%.3f' % f1_score(test_Y, perceptron_pred_test))

0.899 0.885 0.917 0.901 0.860 0.847 0.878 0.862


# SVM

In [16]:
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

clf_svm = LinearSVC(random_state=0, tol=1e-5).fit(train_X, train_Y)
svm_pred_train = clf_svm.predict(train_X)
svm_pred_test = clf_svm.predict(test_X)

print('%.3f' % accuracy_score(train_Y, svm_pred_train),
      '%.3f' % precision_score(train_Y, svm_pred_train),
      '%.3f' % recall_score(train_Y, svm_pred_train),
      '%.3f' % f1_score(train_Y, svm_pred_train),
      '%.3f' % accuracy_score(test_Y, svm_pred_test),
      '%.3f' % precision_score(test_Y, svm_pred_test),
      '%.3f' % recall_score(test_Y, svm_pred_test),
      '%.3f' % f1_score(test_Y, svm_pred_test))

0.936 0.937 0.934 0.936 0.898 0.900 0.895 0.897


# Logistic Regression

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

clf = LogisticRegression(random_state=0).fit(train_X, train_Y)

lg_train_prediction = clf.predict(train_X)
lg_test_prediction = clf.predict(test_X)

print('%.3f' % accuracy_score(train_Y, lg_train_prediction),
      '%.3f' % precision_score(train_Y, lg_train_prediction),
      '%.3f' % recall_score(train_Y, lg_train_prediction),
      '%.3f' % f1_score(train_Y, lg_train_prediction),
      '%.3f' % accuracy_score(test_Y, lg_test_prediction),
      '%.3f' % precision_score(test_Y, lg_test_prediction),
      '%.3f' % recall_score(test_Y, lg_test_prediction),
      '%.3f' % f1_score(test_Y, lg_test_prediction))

0.916 0.919 0.912 0.916 0.900 0.903 0.896 0.899


# Naive Bayes

In [23]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

clf = MultinomialNB()
clf.fit(train_X, train_Y)

nb_train_prediction = clf.predict(train_X)
nb_test_prediction = clf.predict(test_X)

print('%.3f,' % accuracy_score(train_Y, nb_train_prediction),
      '%.3f,' % precision_score(train_Y, nb_train_prediction),
      '%.3f,' % recall_score(train_Y, nb_train_prediction),
      '%.3f,' % f1_score(train_Y, nb_train_prediction),
      '%.3f,' % accuracy_score(test_Y, nb_test_prediction),
      '%.3f,' % precision_score(test_Y, nb_test_prediction),
      '%.3f,' % recall_score(test_Y, nb_test_prediction),
      '%.3f' % f1_score(test_Y, nb_test_prediction))

0.889, 0.895 0.882 0.888 0.873 0.878 0.865 0.871
